In [1]:
import pyvista as pv
import numpy as np
np.random.seed(0)

import forward_growth
import inverse_growth

#### Fast-growing sphere destroys patterns

In [ ]:
# import mesh
filename = 'inputs/sphere.ply'
mesh = pv.read(filename)

mesh = mesh.triangulate()
mesh = mesh.decimate(0.7)
mesh = mesh.triangulate()

# integration params
nx = mesh.n_points
niter = 1000
dt = 0.0001
dx = .002/2 # good for decimated sphere

# RD params
du = 1
dv = 10
g = 1000 # set to zero to just test diffusion
a = 0.126779*0.25
b = 1.1

# filenames
pts_file = 'examples/pts_static_sphere.npy'
laps_file = 'examples/laps_static_sphere.npy'
output_gif = 'examples/static_sphere.gif'


In [ ]:
# grow sphere
forward_growth.grow_forward(mesh, nx, niter, dt, growth_rate=1/dt, grow=True, grow_from_rule=True, pts_output=pts_file, laps_output=laps_file)

In [ ]:
# integrate RD
laps = np.load(laps_file)
u_stored, v_stored = forward_growth.rd_forward(du, dv, g, a, b, nx, dx, niter, dt, laps)

In [ ]:
# plot result
pts = np.load(pts_file)
forward_growth.plot_forward(mesh, u_stored, pts, niter, nskip=2, output_gif=output_gif)


#### Slow-growing sphere allows pattern to form

In [ ]:
# import mesh
filename = 'inputs/sphere.ply'
mesh = pv.read(filename)

mesh = mesh.triangulate()
mesh = mesh.decimate(0.7)
mesh = mesh.triangulate()

# integration params
nx = mesh.n_points
niter = 1000
dt = 0.0001
dx = .002/2 # good for decimated sphere

# RD params
du = 1
dv = 10
g = 1000 # set to zero to just test diffusion
a = 0.126779*0.25
b = 1.1

# filenames
pts_file = 'examples/pts_static_sphere.npy'
laps_file = 'examples/laps_static_sphere.npy'
output_gif = 'examples/static_sphere.gif'


In [ ]:
# grow sphere
forward_growth.grow_forward(mesh, nx, niter, dt, growth_rate=.01/dt, grow=True, grow_from_rule=True, pts_output=pts_file, laps_output=laps_file)

In [ ]:
# integrate RD
laps = np.load(laps_file)
u_stored, v_stored = forward_growth.rd_forward(du, dv, g, a, b, nx, dx, niter, dt, laps)

In [ ]:
# plot result
pts = np.load(pts_file)
forward_growth.plot_forward(mesh, u_stored, pts, niter, nskip=2, output_gif=output_gif)


#### Spots/stripes on static cow

In [13]:
# import mesh
filename = 'inputs/cow.vtp'
mesh = pv.read(filename)

mesh = mesh.triangulate()
mesh = mesh.decimate(0.7)
mesh = mesh.triangulate()

# integration params
nx = mesh.n_points
niter = 1000
dt = 0.0001
dx = .2 # good for decimated cow

# RD params
du = 1
dv = 10
g = 1000 # set to zero to just test diffusion
a = 0.126779*0.25
b = 0.792366

# filenames
data_file = 'examples/data/cow_2.npz'
output_gif = 'examples/cow_2.gif'


In [14]:
# grow sphere
forward_growth.grow_forward(mesh, nx, niter, dt, growth_rate=1/dt, grow=False, grow_from_rule=True, output_file=data_file)



Growth loop completed.


In [15]:
# integrate RD
laps = np.load(data_file)['laps']
u_stored, v_stored = forward_growth.rd_forward(du, dv, g, a, b, nx, dx, niter, dt, laps)

Beginning RD integration loop...
Iteration 1000/1000 (100%)
RD loop completed.


In [16]:
# plot result
pts = np.load(data_file)['pts']
forward_growth.plot_forward(mesh, u_stored, pts, niter, nskip=10, output_gif=output_gif)


Beginning plotting loop...
Iteration 1000/1000 (100%)
Plotting loop completed.
Plotting completed.
